![Insper](https://github.com/danielscarvalho/Insper-DS-Dicas/blob/master/Insper-Logo.png?raw=true)

# Insper Pós-Graduação
## Programa Avançado em Data Science e Decisão [»](https://www.insper.edu.br/pos-graduacao/programas-avancados/programa-avancado-em-data-science-e-decisao/)


# Atividade Integradora
## Setup

### Dependencias

In [ ]:
import pandas as pd
import numpy as np
from dfply import *
import altair as alt
import missingno as msno

### Carregamento dos Dados

In [ ]:
data = pd.read_csv("cs_bisnode_panel.csv")
dicionario_de_dados_0 = pd.read_excel("bisnode_variable_names.xls", header=4)

In [ ]:
data.head()

## Limpeza dos Dados

### Dicionário de dados - Limpeza

Ao carregar o dicionário de dados a primeira coluna pega seu nome da quarta linha da tabela (argumento `header=4` acima). As outras três colunas são nomeadas abaixo.

In [ ]:
dicionario_de_dados_1 = dicionario_de_dados_0.rename({'Unnamed: 1': 'description',
                                                      'Unnamed: 2': 'type',
                                                      'Unnamed: 3': 'footnote'},
                                                     axis=1)

In [ ]:
dicionario_de_dados_1.columns

Então retiramos as linhas não relevantes para a analize, incluindo linhas totalmente em branco e uma linha com informação de versão da base de dados: 
 - `v 0.92. 2021-02-04`

In [ ]:
dicionario_de_dados = dicionario_de_dados_1\
                       .drop(index=54)\
                       .dropna(how="all")\
                       .reset_index()\
                       .drop('index', axis='columns')

In [ ]:
dicionario_de_dados.sample(5)

### Removendo colunas específicas:

In [ ]:
columns_to_remove = ['COGS', 'finished_prod', 'net_dom_sales', 'net_exp_sales', 'wages', 'D']

data.drop(columns=columns_to_remove, inplace=True)

data.columns

### Removendo dados do ano 2016:

Registros que começaram ou terminaram (ocorreram em) 2016 são removidos do conjunto.

In [ ]:
colunas_data = ["begin", "end", "founded_date", "exit_date"]

data.dtypes[colunas_data]

Dados destas colunas precisam estar em formato de datetime para serem operados corretamente, então vamos converte-los.

In [ ]:
for column in colunas_data:
    data[column] = pd.to_datetime(data[column], format="%Y-%m-%d")

data[colunas_data].dtypes

Removendo Registros do ano de 2016:

In [ ]:
data = data[data["year"]!=2016]
data["year"].unique()

### Criando coluna para variável "Dependente":
---

- Vamos operar com o conceito de "atividade", empresas que não tiveram atividade por mais de 2 anos são consideradas "inativas".

Obs.: ativa = 0, inativa = 1

In [ ]:
# Sort the DataFrame by company and year
data.sort_values(by=['comp_id', 'year'], inplace=True)

# Create a shifted column to check sales in X + 2 years
data.loc[data['comp_id'] == data['comp_id'].shift(-1), 'sales_x1'] = data['sales'].shift(-1)
data.loc[data['comp_id'] == data['comp_id'].shift(-2), 'sales_x2'] = data['sales'].shift(-2)

# Create a condition to identify companies that ceased to operate 
# (sem vendas por mais de 2 anos)
condition = (data['sales_x1'] == 0) & (data['sales_x2'] == 0)

# Create a new 'dependente' column with 1 for ceased companies and 0 otherwise
data['dependente'] = condition.astype(int)


Vamos observar as duas primeiras empresas da dataframe para averiguar seus estados de "dependencia":

In [ ]:
# lista de colunas para avaliação:
check_list=["comp_id", "year", "sales", "sales_x1", "sales_x2", "dependente"]

data[check_list].head(18)

Vamos averiguar agora os anos em que as empresas tiveram atividade e inatividade:

In [ ]:
data_grouped = data.groupby('comp_id')

#Contando anos de acompanhamento
comp_years = data_grouped['year'].count()

#Contando anos de "atividade"
active_years = data_grouped.apply(lambda group: (group['sales'] > 0).sum())

#contando anos de "inatividade"
inactive_years = data_grouped.apply(lambda group: (group['sales'] == 0).sum())

#Contando anos de "dependência"
depend_years = data_grouped.apply(lambda group: (group['dependente'] == 1).sum())

In [ ]:
activity_df = pd.DataFrame({"Total years":comp_years, "Active years":active_years, 
                            "Inactive years":inactive_years, "Dependent years":depend_years})

activity_df